In [2]:
import pymongo
import pandas
import numpy as np
from sklearn.neighbors import NearestNeighbors
# 设置精度
np.set_printoptions(precision=3)

In [3]:
# mongodb uri
conn_str = "mongodb://1.15.118.125:27017/"
# 客户端连接
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
collection_user = client.get_database("Lab").get_collection("Mission")

In [4]:
# 定义函数用

In [5]:
# 查询 title 与对应的 tags
title_with_tags = {}
for doc in collection_user.find():
    mission_tags = doc.get("missionTags")
    mission_title = doc.get("title")
    title_with_tags[mission_title] = mission_tags

# pands 格式化输出
ser_tag_tags = pandas.Series(title_with_tags)
ser_tag_tags.head(len(ser_tag_tags))

学生骨干成长训练营举办暑期专题培训                     [学生工作, 培训会]
新生教师班主任工作研讨会召开                    [教学建设, 研讨会, 迎新]
2022级本科生线上新生家长会                    [师生交流, 会议, 迎新]
2022级研究生新生入学教育活动                       [迎新, 主题教育]
研究生班级新生见面会顺利举行                         [迎新, 班级活动]
                                       ...       
管理学院第二届工商管理学科高端论坛系列活动: 学科建设座谈会        [教学建设, 交流会]
新进老师培训                                [教学建设, 培训会]
学生骨干训练营的结营仪式                           [学生工作, 仪式]
十二月舍导交流                              [师生交流, 校园生活]
中国管理学前沿研究研讨会: 开幕式与主报告             [学术, 仪式, 汇报/报告]
Length: 101, dtype: object

In [6]:
# 查询 tag 的频次
def get_tag_times(dict_value_tags):
    tag_with_time = {}
    for key in dict_value_tags:
        for tag in dict_value_tags[key]:
            if tag not in tag_with_time:
                tag_with_time[tag] = 1
            else:
                tag_with_time[tag] += 1
    return tag_with_time


# pands 格式化输出
ser_tag_with_time = pandas.Series(get_tag_times(title_with_tags))
ser_tag_with_time.head(len(ser_tag_with_time))

学生工作      4
培训会       6
教学建设     21
研讨会       2
迎新        8
师生交流     10
会议       13
主题教育      2
班级活动      8
就业发展      6
校园生活      5
党团活动     27
社会实践      2
调研活动      1
交流会      11
奖学金       4
宣讲会       6
其他活动      6
文体活动     10
赛事        9
仪式        5
学术       10
讲座       10
趣味活动      2
答辩        1
生活会       1
采访        1
课程学习      5
汇报/报告     4
dtype: int64

In [7]:
# 定义 0 矩阵
dist = np.zeros((len(title_with_tags), len(get_tag_times(title_with_tags)),))
# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
# 获取 title 的 list
title_list = list(title_with_tags.keys())

for mission_index in range(len(title_with_tags)):
    for tag in title_with_tags[title_list[mission_index]]:
        for tag_index in range(len(tags_list)):
            if tag == tags_list[tag_index]:
                dist[mission_index][tag_index] = 1

In [8]:
# 使用 KNN 算法计算样本相互距离
calcu_nbr = NearestNeighbors(n_neighbors=12, algorithm='ball_tree').fit(dist)
distances,indices = calcu_nbr.kneighbors(dist)

In [9]:
# 输出距离
pandas.DataFrame(distances)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.000000,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.732051,1.732051,1.732051,1.732051
1,0.0,1.000000,1.414214,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051
2,0.0,1.000000,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051
3,0.0,0.000000,1.414214,1.414214,1.414214,1.414214,1.732051,1.732051,1.732051,1.732051,1.732051,1.732051
4,0.0,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.732051
...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,1.000000,1.000000,1.000000,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214
97,0.0,0.000000,1.000000,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214
98,0.0,1.000000,1.414214,1.414214,1.414214,1.414214,1.414214,1.732051,1.732051,1.732051,1.732051,1.732051
99,0.0,0.000000,1.000000,1.000000,1.000000,1.414214,1.414214,1.732051,1.732051,1.732051,1.732051,1.732051


In [10]:
# 输出邻近的 index 值
print(indices)

[[ 79   0  98 ...  45  50  44]
 [  1  17  45 ...   3   4  29]
 [  2  28  14 ...   3   4  29]
 ...
 [ 98  44  88 ...  50  61  75]
 [ 99  94  42 ...  38  37  44]
 [100  95  44 ...  84  87  19]]


In [11]:
# 根据矩阵进行寻找
def find_similar_mission(title_str,num):
    similar_mission_list = []
    my_index = title_list.index(title_str)
    print("为 " + title_str+" 寻找相似任务")
    pointer = 0
    count = 0
    while count < num:
        if indices[my_index][pointer] == my_index:
            pointer += 1
            continue
        else:
            similar_mission_list.append(title_list[indices[my_index][pointer]])
            count += 1
            pointer += 1
    for mis in similar_mission_list:
        print(" -> "+mis)
    return similar_mission_list

In [12]:
get_num = 5
find_similar_mission('我院召开国际认证工作会议',get_num)
find_similar_mission('开展9月党员舍导进宿舍活动',get_num)
find_similar_mission('“廖建桥教室”揭牌仪式活动',get_num)
print("done")

为 我院召开国际认证工作会议 寻找相似任务
 -> 领学团第一次集体备课会
 -> AOL第三轮工作会议
 -> 2022年度秋季学期教职工大会举行
 -> 我院开展EQUIS认证资质预评估
 -> 2022级本科生入学教育大会暨年级大会顺利召开
为 开展9月党员舍导进宿舍活动 寻找相似任务
 -> 开展10月党员舍导进宿舍活动
 -> 2022年“红色领航员”党员舍导进宿舍活动
 -> 十一月舍导交流
 -> 十二月舍导交流
 -> 本科第五党支部开展10月主题党日活动
为 “廖建桥教室”揭牌仪式活动 寻找相似任务
 -> 我院开展EQUIS认证资质预评估
 -> 举行“喻见·爱”捐赠答谢仪式
 -> 课堂进企业 | 劲牌有限公司：探索数字化营销之路
 -> 华中科技大学2022级MBA开学典礼暨入学导航
 -> 我院召开国际认证工作会议
done
